# How to setup Zipline backtester with custom data from FXCM  

## 1. Install standard packages
ou should be able to install zipline by using pip. If it doesn't work, there are other ways to install zipline that you can look up.  

```
pip install zipline
```
I had to upgrade some packages
```
pip install -U pip
pip install -U numpy
```

## 2. Download custom forex calendar  
Zipline has made its calendars a standalone package available at https://github.com/quantopian/trading_calendars  
to make it more available for adding custom calendars, which is what you want because Zipline doesn't support the 24/7 forex calendar as default.  

Clone my fork of `trading_calendars` into any preferable location  
```
git clone https://github.com/grananqvist/trading_calendars.git  
```
This fork as a new calendar called `forex`  

## 3.  Download my zipline utilities  
Necessary utilities are located in zipline_extensions in this repo, download them by cloning the repo into preferable location  
```
git glone https://github.com/grananqvist/Machine-Learning-Trading-Strategies.git
```

## 4. Add packages to PYTHONPATH
For python to find the repos above, add them to your `$PYTHONPATH` 
```
export PYTHONPATH=$PYTHONPATH:<path-to-trading_calendars>:<path-to-trading-repo>
```
In my case, my `$PYTHONPATH` looks like this  
`/Users/system/Github/Machine-Learning-Trading-Strategies/:/Users/system/Github/trading_calendars/`  

Test if packages are installed correctly (you will need to restart the jupyter notebook kernel):  

In [13]:
from zipline.utils.calendars import get_calendar
get_calendar('forex')
from zipline_extensions.bundles.fxcm import via_fxcm_csv_daily

## 5. Download FXCM data  
1. Clone my FXCM data downloader
```
git clone https://github.com/grananqvist/FXCM-Forex-Data-Downloader.git
```
2. Get an [API key](https://www.fxcm.com/uk/algorithmic-trading/api-trading/) from FXCM, works with a demo account  
3. Download data using the download script  
```
python main.py -pe D1 -t <YOUR FXCM API KEY> -p <ANY LOCATION TO STORE DATA>
```

## 6. Edit your extensions.py
A file called `extensions.py` should be located in `~/.zipline/`  
Insert the code below into `extensions.py` to register fxcm_daily as a bundle for zipline  

**Note:** replace `DATA_PATH` with the path to your downloaded dataset  

In [ ]:
import os
from zipline.data.bundles import register
from zipline_extensions.bundles.fxcm import via_fxcm_csv_daily
DATA_PATH = os.path.join(
    os.environ['HOME'], 'Github/Machine-Learning-Trading-Strategies/data/D1/')

register('fxcm_daily', via_fxcm_csv_daily(DATA_PATH), calendar_name='forex')

## 7. Ingest bundle

In [16]:
#%load_ext zipline
!zipline ingest -b fxcm_daily

Making bundle with symbols:  ('AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'AUS200', 'Bund', 'CADCHF', 'CADJPY', 'CHFJPY', 'CHN50', 'Copper', 'ESP35', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURNOK', 'EURNZD', 'EURSEK', 'EURTRY', 'EURUSD', 'EUSTX50', 'FRA40', 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 'GBPNZD', 'GBPUSD', 'GER30', 'HKG33', 'JPN225', 'NAS100', 'NGAS', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'SOYF', 'SPX500', 'TRYJPY', 'UK100', 'UKOil', 'US30', 'USDCAD', 'USDCHF', 'USDCNH', 'USDHKD', 'USDJPY', 'USDMXN', 'USDNOK', 'USDOLLAR', 'USDSEK', 'USDTRY', 'USDZAR', 'USOil', 'XAGUSD', 'XAUUSD', 'ZARJPY')
symbol= AUDCAD file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/AUDCAD_D1.csv
--Preprocessing summary--
candles before: 4826
candles after: 4433
symbol= AUDCHF file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/AUDCHF_D1.csv
--Preprocessing summary--
candles before: 4770
candles after: 4376
symbol= AUDJPY file= /Users/system/Gith

--Preprocessing summary--
candles before: 4392
candles after: 4432
symbol= USDCAD file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDCAD_D1.csv
--Preprocessing summary--
candles before: 4829
candles after: 4433
symbol= USDCHF file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDCHF_D1.csv
--Preprocessing summary--
candles before: 4823
candles after: 4433
symbol= USDCNH file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDCNH_D1.csv
--Preprocessing summary--
candles before: 1935
candles after: 1708
symbol= USDHKD file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDHKD_D1.csv
--Preprocessing summary--
candles before: 3271
candles after: 3032
symbol= USDJPY file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDJPY_D1.csv
--Preprocessing summary--
candles before: 4833
candles after: 4433
symbol= USDMXN file= /Users/system/Github/Machine-Learning-Trading-Strategies/data/D1/USDMXN_D1.cs